# Projeto de processador simplificado

Author: Petrúcio Medeiros

- Descrição: Neste projeto foi considerado um contador de programa (PC), uma memória de instruções que contém 100 endereços e cada endereço com 32 bits. Um banco de registradores com 32 endereços de 32 bits previamente definido com os endereços 0 e 1 preenchidos com os valores 3 e 5, respectivamente. Uma unidade lógica aritmética (ULA) para realizar a operação de soma. Finalmente, uma Unidade Central de Processamento (CPU) para coordenar as operações entre essas unidades.

  As instruções são codificadas como no MIPS 32:
  - Opcode : 6 bits
  - Registrador destino : 5 bits
  - Registrador fonte 1 : 5 bits
  - Registrador fonte 2 : 5 bits
  - Shamp ( operações de deslocamento ) : 5 bits
  - Funct ( variacoes das operacoes especificadas do opcode ) : 6 bits

[x] Modificar o código do processador proposto na última semana para que a leitura de instruções e dados dos registradores seja realizada apartir de um arquivo .txt. <br>
[x] Utilizar a codificação one-hot para definir instruções lógicas na ULA do processador simplificado. <br>
[x] Criar um testbench para o processador simplificado (Obs.: A única variável de entrada do processador é o clock, visto que as instruções e valores dos registradores estão sendo enviadas por meio de arquivos .txt)

In [63]:
# biblioteca do verilog
%%bash
sudo apt-get update
sudo apt-get install verilog gtkwave

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Rea

In [64]:
%%file pc.sv
// Contador de programa ( PC )
module pc (clk, address);
  // Declaracao de portas
  input clk;
  reg [31:0] counter;
  output [31:0] address;

  // Iniciando endereco
  initial begin
    counter = -32'd4;
  end

  // Funcionamento do contador => contador + 1
  always @(posedge clk) begin
    counter <= counter + 32'd4;
  end

  // Atribuindo o valor do endereco
  assign address = counter;
endmodule

Overwriting pc.sv


In [65]:
%%file instructions.txt
00000100
01000000
00001000
00000000

Overwriting instructions.txt


In [66]:
%%file instructions_memory.sv

`define NULL 0

// Memória de instrucoes
module instructions_memory ( clk, counter, output_instruction );
  // Declaracao de portas
  input clk;
  input [31:0] counter;
  output [31:0] output_instruction;

  // Criando uma memoria de instrucoes com 4 enderecos de 8 bits (1 byte)
  //reg [31:0] memoriaInstrucoes [99:0];
  reg [7:0] memoriaInstrucoes [99:0];

  /*
  // Iniciando a memoria de instrucoes
  initial begin
    // 6 bits ( opcode ) = 000001
    // 5 bits ( registrador destino ) = 00010
    // 5 bits ( registrador origem 1 ) = 00000
    // 5 bits ( registrador origem 2 ) = 00001
    // 5 bits ( shamt - operacoes de deslocamento ) = 00000
    // 6 bits ( funct - variacoes das operacoes especificadas do opcode ) = 00000
    // memoriaInstrucoes[8'd0] = 32'b00000100010000000000100000000000;
    memoriaInstrucoes[8'd0] = 8'b00000100;
    memoriaInstrucoes[8'd1] = 8'b01000000;
    memoriaInstrucoes[8'd2] = 8'b00001000;
    memoriaInstrucoes[8'd3] = 8'b00000000;
  end
  */

  initial begin
    // Abrindo arquivo
    f = $fopen("instructions.txt","r");
    // Se arquivo nao contem nenhuma informacao
    if ( f == `NULL ) begin
      $display("Arquivo sem instrucoes");
      $finish;
    end

    while (! $feof( f )) begin
      r = $fscanf( f,"%8b\n", data);
      memoriaInstrucoes[counterMemory] = data;
      counterMemory = counterMemory + 32'd4;
    end
    
    // Fechando arquivo
    $fclose( f );
  end

  // Atribuindo o valor da instrucao
  assign output_instruction = {memoriaInstrucoes[ counter + 0 ],
                               memoriaInstrucoes[ counter + 1 ],
                               memoriaInstrucoes[ counter + 2 ],
                               memoriaInstrucoes[ counter + 3 ]};
endmodule

Overwriting instructions_memory.sv


In [67]:
%%file registers.txt
8
10

Overwriting registers.txt


In [69]:
%%file register_bank.sv
`define NULL 0

// Banco de registradores
module register_bank ( clk, addr_regd, addr_reg1, addr_reg2, data_in, value_regd, value_reg1, value_reg2 );
  // Declaracao de portas
  input clk;
  input [4:0] addr_regd, addr_reg1, addr_reg2;
  input [31:0] data_in;
  output [31:0] value_regd, value_reg1, value_reg2;

  integer f, r; // arquivo
  reg [31:0] data = 32'd0;
  integer counterRegister = 5'd0;

  // Criando um banco de registradores com 32 enderecos de 32 bits
  reg [31:0] registerBank [31:0];


  // Iniciar valores para os registradores reg1 e reg2 no banco de registradores
  initial begin
    // Abrindo arquivo
    f = $fopen("/content/registers.txt","r");
    // Se arquivo nao contem nenhuma informacao
    if ( f == `NULL ) begin
      $display("Arquivo sem dados");
      $finish;
    end 

    while (! $feof( f )) begin
      r = $fscanf( f,"%32d\n", data);
      registerBank[counterRegister] = data;
      counterRegister = counterRegister + 5'd1;
    end

    // Fechando arquivo
    $fclose( f );
  end

  // Descrevendo o comportamento de escrita no banco de registradores
  always @(posedge clk) begin
    registerBank[addr_regd] <= data_in; 
  end

    // Atribuicao de valores
  assign value_regd = registerBank[addr_regd];
  assign value_reg1 = registerBank[addr_reg1];
  assign value_reg2 = registerBank[addr_reg2];
endmodule

Overwriting register_bank.sv


In [70]:
%%file ula.sv
// Unidade logica aritmetica (ULA)
module ula ( opcode, in1, in2, out );
  // Declaracao de portas
  input [5:0] opcode;
  input [31:0] in1, in2;
  output [31:0] out;
  
  // Variavel para guardar resultado
  reg [31:0] result;

  // Descrevendo o comportamento da ULA
  always @(in1, in2, opcode) begin
    case ({opcode})
      6'b000001 : result = in1 + in2;
      6'b000010 : result = in1 - in2;
      6'b000100 : result = in1 * in2;
      6'b001000 : result = in1 / in2;
      default : result = 32'd0;
    endcase
  end

  // Atribuindo o valor do resultado
  assign out = result;
endmodule

Overwriting ula.sv


In [71]:
%%file cpu.v

`include "pc.sv"
`include "instructions_memory.sv"
`include "register_bank.sv"
`include "ula.sv"

// Unidade Central de Processamento
module cpu ( clock, value_output );
  // Declaracao de portas
  input clock;
  output [31:0] value_output;

  // Variaveis intermediarias
  wire [31:0] address, instruction, data_ula, value_regd, value_reg1, value_reg2;

  // PC
  pc p(clock, address);

  // Acessando memoria de instrucoes
  instructions_memory im( clock, address, instruction );

  wire [4:0] regd, reg1, reg2;
  assign regd = instruction[25:21];
  assign reg1 = instruction[20:16];
  assign reg2 = instruction[15:11];

  wire [5:0] op;
  assign op = instruction[31:26];

  // Acessando banco de registradores
  register_bank rb( clock, regd, reg1, reg2, data_ula, value_regd, value_reg1, value_reg2 );

  // Operando sobre os valores dos registradores
  ula alu( op, value_reg1, value_reg2, data_ula );

  assign value_output = data_ula;

  /*

  // Atribuindo o valor do registrador destino
  assign value_output = value_regd;
  */

endmodule

/*

 Módulo de teste

*/
module teste();               // Definindo um módulo de teste, onde adiciono valores
parameter nbits = 9;          // 2^{entradas} = 2^{1} = 2
reg counter;                  // Criando um registro de 2 bits para entradas
wire [31:0] z;                // Declarando um fio de saída
integer k;                    // Declarando um inteiro para percorrer todas as possibilidades
	cpu t(counter, z);
	initial begin               // // Início do bloco de comandos
		$display("clk | z");      // printf apenas com string
		$monitor(" %1b  | %1d", counter, z[31:0]); // printf passando variáveis
    counter = 0;
    /*
    Percorre a quantidade de saídas e armazena no contador
    */
		for (k=1; k<nbits; k=k+1)
     #1 counter = k;
		$finish;
	end // Fim do bloco de comandos
endmodule


Overwriting cpu.v


In [72]:
%%bash
iverilog cpu.v -o cpu
vvp cpu

clk | z
 0  | x
 1  | 11


./instructions_memory.sv:34: error: Could not find variable ``f'' in ``teste.t.im''
./instructions_memory.sv:36: error: Unable to bind wire/reg/memory `f' in `teste.t.im'
./instructions_memory.sv:36: error: Unable to elaborate condition expression.
./instructions_memory.sv:41: error: Unable to bind wire/reg/memory `f' in `teste.t.im'
./instructions_memory.sv:42: error: Could not find variable ``r'' in ``teste.t.im''
./instructions_memory.sv:43: error: Unable to bind wire/reg/memory `counterMemory' in `teste.t.im'
./instructions_memory.sv:43: error: Unable to bind wire/reg/memory `data' in `teste.t.im'
./instructions_memory.sv:44: error: Could not find variable ``counterMemory'' in ``teste.t.im''
./instructions_memory.sv:48: error: Unable to bind wire/reg/memory `f' in `teste.t.im'
9 error(s) during elaboration.
